# Alt‑Text Generation Playground
Interactive notebook to explore and run `main.py`.

**What you can do here**
1. Set your `OPENROUTER_API_KEY`.
2. Inspect and tweak `MODELS` and `MEDIA_IDS` at runtime.
3. Generate prompts for selected media.
4. Execute the full pipeline and inspect outputs.
5. Load the results table for quick analysis.

Notes:
- The notebook expects `main.py` in the same directory.
- Network calls occur in `fetch_and_save_metadata` and the OpenRouter API.
- You can override `MODELS` and `MEDIA_IDS` without editing `main.py`.

## 0) Environment and dependencies

In [ ]:
# Optional: install or update dependencies in the current kernel
# Uncomment if needed.
# %pip install -q -U python-dotenv pydantic requests pandas pyarrow


In [ ]:
# Provide the OpenRouter API key as an environment variable for this session
import os
from getpass import getpass

if not os.getenv("OPENROUTER_API_KEY"):
    os.environ["OPENROUTER_API_KEY"] = getpass("Enter OPENROUTER_API_KEY: ")
print("OPENROUTER_API_KEY set:", bool(os.getenv("OPENROUTER_API_KEY")))

## 1) Import `main.py`
The notebook assumes the script file is named **`main.py`**.

In [ ]:
# If importing fails, use %run as a fallback to load the module definitions into the namespace
import importlib, sys, pathlib

base = pathlib.Path().resolve()
assert (base / "main.py").exists(), "main.py not found next to this notebook."

try:
    import main  # noqa: F401

    importlib.reload(main)
except Exception as e:
    print("Import failed, executing %run main.py to populate symbols:", e)
    %run -i main.py
    import main

    importlib.reload(main)

print("Imported main from:", main.__file__)

## 2) Inspect current configuration

In [ ]:
print("OPENROUTER_URL:", getattr(main, "OPENROUTER_URL", None))
print("METADATA_URL:", getattr(main, "METADATA_URL", None))
print("TIMEOUT_SECONDS:", getattr(main, "TIMEOUT_SECONDS", None))
print("RUNS_DIR:", getattr(main, "RUNS_DIR", None))
print("MODELS:", getattr(main, "MODELS", None))
print("MEDIA_IDS:", getattr(main, "MEDIA_IDS", None))

## 3) Override configuration at runtime
Adjust `MODELS` and `MEDIA_IDS` without editing the script.

In [ ]:
# Example: narrow to one model and one media id during experimentation.
# Comment or edit as you wish.
main.MODELS = [
    "google/gemini-2.5-flash-lite",
    # "mistralai/pixtral-12b",
    # "openai/gpt-4.1-nano",
    # "allenai/molmo-7b-d",
]
main.MEDIA_IDS = ["m93036"]

print("MODELS ->", main.MODELS)
print("MEDIA_IDS ->", main.MEDIA_IDS)

## 4) Quick prompt preview for a media id
This lets you see the constructed prompt and image URL before executing a full run.

In [ ]:
# Fetch metadata JSON and build a prompt for the first media id
payload = main.fetch_and_save_metadata(main.METADATA_URL, main.mk_run_dir())
db = main.load_db_from_payload(payload)

mid = main.MEDIA_IDS[0]
media = db[mid]
prompt = main.build_prompt(media)
message = main.build_messages(prompt, str(media.object_thumb))

print("Media:", mid)
print("\n=== Prompt ===\n")
print(prompt)
print("\n=== Image URL ===\n", media.object_thumb)
print(
    "\n=== Message payload snippet ===\n", message[0]["content"][0] if message else {}
)

## 5) Execute the full pipeline
Runs `main.main()` which will fetch metadata, save a copy for the run, call models, and persist outputs.

In [ ]:
main.main()
print("Run complete. See the generated runs/ timestamped directory.")

## 6) Load and inspect the latest results table

In [ ]:
import pandas as pd, pathlib, re

runs_dir = pathlib.Path(getattr(main, "RUNS_DIR", "runs"))
assert runs_dir.exists(), "runs/ directory not found. Did you run the pipeline?"

# Find newest run directory
run_dirs = sorted(
    [p for p in runs_dir.iterdir() if p.is_dir()],
    key=lambda p: p.stat().st_mtime,
    reverse=True,
)
latest = run_dirs[0]
print("Latest run:", latest)

# Find the CSV for that run
tables = sorted(latest.glob("alt_text_runs_*.csv"))
assert tables, "No results CSV found in the latest run."
csv_path = tables[-1]
print("Results table:", csv_path)

df = pd.read_csv(csv_path)
print("Rows:", len(df), "Columns:", len(df.columns))
df.head(3)

## 7) Compare model outputs for one media id

In [ ]:
from pprint import pprint

row = df.iloc[0].to_dict()
objectid = row.get("objectid", "<unknown>")
print("Object ID:", objectid)

# Collect model-specific content columns
model_cols = [c for c in df.columns if c.endswith("__content")]
pairs = []
for c in model_cols:
    model_name = c.replace("__content", "").replace("__", "/")
    pairs.append((model_name, row.get(c)))

for name, content in pairs:
    print("\n###", name, "\n", content)

## 8) Optional: run a single model on one media id
Calls the underlying request function directly for quick checks.

In [ ]:
mid = main.MEDIA_IDS[0]
payload = main.fetch_and_save_metadata(main.METADATA_URL, main.mk_run_dir())
db = main.load_db_from_payload(payload)
media = db[mid]
messages = main.build_messages(main.build_prompt(media), str(media.object_thumb))

model = main.MODELS[0]
resp = main.call_openrouter(
    api_key=os.environ["OPENROUTER_API_KEY"], model=model, messages=messages
)
print("Provider:", resp.provider, "| Model:", resp.model, "| Completion id:", resp.id)

choice = resp.choices[0] if resp.choices else None
print("\nAlt text:")
print(choice.message.content if choice else "<no content>")